In [1]:
import sys
sys.path.insert(0, 'C:/Users/chris/Documents/GitHub/AzureFunctions/AzureFunctions')
import datetime
import logging
import time
from datetime import date
import os
from io import BytesIO
import pandas as pd

from azure.storage.filedatalake import DataLakeServiceClient
import azure.functions as func
from shared_code import utils

In [2]:
azure_utils = utils.AzureUtils()
secret_client = azure_utils.initialize_key_vault()

sa_secret = azure_utils.get_key_vault_secret(secret_client, 'sa-secret')
sa_name = azure_utils.get_key_vault_secret(secret_client, 'sa-name')

azure_utils.initialize_storage_account_ad(sa_secret.value, sa_name.value)
azure_utils.initialize_data_lake(sa_name.value, sa_secret.value)

In [ ]:
df_raw = azure_utils.ingest_bronze_data(f"fundamentals/{date.today()}")
df_raw.head()

In [3]:
selected_cols = [
    'Ticker',
    'ObservationDate',
    'mostRecentQuarter',
    'fullTimeEmployees',
    'ebitdaMargins',
    'profitMargins',
    'grossMargins',
    'operatingCashflow',
    'revenueGrowth',
    'operatingMargins',
    'ebitda',
    'targetLowPrice',
    'recommendationKey',
    'grossProfits',
    'freeCashflow',
    'targetMedianPrice',
    'currentPrice',
    'earningsGrowth',
    'currentRatio',
    'returnOnAssets',
    'numberOfAnalystOpinions',
    'targetMeanPrice',
    'debtToEquity',
    'returnOnEquity',
    'targetHighPrice',
    'totalCash',
    'totalDebt',
    'totalRevenue',
    'recommendationMean',
    'enterpriseToRevenue',
    'enterpriseToEbitda',
    '52WeekChange',
    'morningStarRiskRating',
    'forwardEps',
    'sharesOutstanding',
    'heldPercentInstitutions',
    'trailingEps',
    'lastDividendValue',
    'priceToBook',
    'heldPercentInsiders',
    'beta',
    'enterpriseValue',
    'priceToSalesTrailing12Months',
    'forwardPE',
    'trailingPE',
    'marketCap'
]
cols_to_exclude = ['Ticker', 'ObservationDate', 'recommendationKey', 'mostRecentQuarter']

dataCleaning_utils = utils.DataCleaning()

df_cleaning = (
    azure_utils.ingest_bronze_data(f"fundamentals/{date.today()}")
    .pipe(dataCleaning_utils.pivot_fundamentals_dataframe)
    .pipe(dataCleaning_utils.select_dataframe_columns, selected_cols)
    .pipe(dataCleaning_utils.set_dtype_to_numeric, cols_to_exclude)
    .pipe(dataCleaning_utils.change_timestamp_to_datetime, "mostRecentQuarter")
    .pipe(dataCleaning_utils.change_timestamp_format, "ObservationDate")
)

df_cleaning.head()

Attribute,Ticker,ObservationDate,mostRecentQuarter,fullTimeEmployees,ebitdaMargins,profitMargins,grossMargins,operatingCashflow,revenueGrowth,operatingMargins,...,trailingEps,lastDividendValue,priceToBook,heldPercentInsiders,beta,enterpriseValue,priceToSalesTrailing12Months,forwardPE,trailingPE,marketCap
0,8TRA.ST,2022-10-27,2022-06-30,99865.0,0.09657,0.02293,0.16506,-643000000.0,0.336,0.02887,...,17.65,0.5,4.97931,0.89722,1.760954,89272287232.0,2.063978,NaN,8.181303,72199995392.0
1,AAK.ST,2022-10-27,2022-09-30,4000.0,0.0,0.0,0.0,NaN,NaN,0.0,...,6.13,2.5,3.627528,0.30535,NaN,41670291456.0,NaN,19.77871,26.9739,42918084608.0
2,ABB.ST,2022-10-27,2022-09-30,106380.0,0.15502,0.13646,0.32536,1620000000.0,0.054,0.12882,...,22.88,0.82,48.985092,0.00037,1.041506,580004478976.0,19.99569,207.31543,13.500875,583654178816.0
3,ABLI.ST,2022-10-27,2022-06-30,8.0,0.0,0.0,0.0,-119588000.0,NaN,-812.4888,...,-0.313,NaN,0.467227,0.04307,0.930252,-55321620.0,1329.3208,NaN,NaN,176799664.0
4,ACAD.ST,2022-10-27,2022-09-30,13982.0,0.20809,0.04122,0.30712,2779000064.0,0.066,0.08664,...,5.43,1.75,0.81756,0.22875,1.066324,15531668480.0,0.355533,5.921363,8.320442,5166604288.0


In [ ]:
cols_company = [
    'zip', 
    'sector', 
    'longBusinessSummary',
    'city',
    'country', 
    'website',
    'address1',
    'industry',
    'longName',
    'financialCurrency',
    'exchange',
    'isEsgPopulated',
    'quoteType'
]
df_company = df_raw[(df_raw.Attribute.isin(cols_company)) & (df_raw.ObservationDate == str(date.today()))]
df_company = df_company.pivot(index = "Ticker", columns = "Attribute", values = "Recent")
df_company.head()

In [ ]:
parquet_file = df_company.to_parquet(index = False)
utils.upload_blob(parquet_file, f"gold/companies", "dim_company.parquet")

In [ ]:
df_company = utils.download_parquet_blob(f"gold/companies", f"dim_company.parquet")
df_company.head()

In [ ]:
# pd.set_option('display.max_rows', None)
# pd.set_option('display.max_columns', None)
pd.DataFrame({"Data Type" : df_data.dtypes})

In [ ]:
pd.DataFrame({"Missing Values [%]" : df_data.isna().sum() / len(df_data.index)*100})

In [ ]:
parquet_file = df_data.to_parquet(index = False)
utils.upload_blob(parquet_file, f"gold/financials", "fact_financials.parquet")

In [ ]:
df_financials = utils.download_parquet_blob(f"gold/financials", f"fact_financials.parquet")
df_financials.head()

In [ ]:
data1 = {
  "name": ["Sally", "Mary", "John"],
  "age": [50, 40, 30]
}

data2 = {
  "name": ["Sally", "Peter", "Micky"],
  "age": [77, 44, 22]
}

df1 = pd.DataFrame(data1).set_index("name")
df2 = pd.DataFrame(data2).set_index("name")

pd.concat([df1[~df1.index.isin(df2.index)], df2])